### EWSR1 motif from [JASPAR](http://jaspar.genereg.net/cgi-bin/jaspar_db.pl?ID=MA0149.1&rm=present&collection=CORE)
![title](http://jaspar.genereg.net/static/logos/MA0149.1.BIG.png)

In [1]:
import numpy as np

seq_length = 40
num_sample = 1000

# PFM from JASPAR
motif = np.array([[   0,   2, 104, 104,   1,   2, 103, 102,   0,   0,  99, 105,   0,   0, 100, 102,   5,   3],
                  [   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   4,   0,   0,   2,   3,   0,   0,   3],
                  [ 105, 103,   1,   1, 104, 102,   2,   3, 104, 103,   2,   0, 105, 103,   0,   2,  97,  97],
                  [   0,   0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   0,   0,   0,   2,   1,   3,   2]])

In [2]:
freq = np.hstack([np.ones((4,(seq_length-motif.shape[1])//2)), 
                  motif,
                  np.ones((4,(seq_length-motif.shape[1])//2))])
print('Sequence matrix shape: {}'.format(freq.shape))

Sequence matrix shape: (4, 40)


In [3]:
#normalize to PWM and generate positive samples
pos = np.array([np.random.choice(['A', 'C', 'G', 'T'], num_sample, p=freq[:,i]/sum(freq[:,i])) 
                for i in range(seq_length)]).transpose()
[''.join(x) for x in pos[1:10,:]]

['GGTAACCAACAGGAAGGAAGGAAGCAAGGGTTACATGTTG',
 'GGCTAGTAGTAGGAAGGAAGGAAGGAAGGAACCTTCATTC',
 'GGGTTATAGTAGGAAGGAAGGAAGGAAGGAGTACTCAGAT',
 'GCATTCTTATTGGAAGGAAGGGAGGAAGGGTTGGAAATCA',
 'GGCAGGTGAGAGGAAGGAAGGAAGGAAGGCGAAGTTAGTC',
 'CTTTTTTTTCGGGAAGGAAGGAAGGAAGGGAAACGGGCTG',
 'TTGTATTGCAAGGAAGGAAGGAAGGAAGGGAATCTCTGCA',
 'GAACCGTCAAGGGAAGGAAGGAAGGAAGGAGGGAGCCCAG',
 'CGCGGCAAATGGGAAGGAGGGAAGGAAGGCTAATGGAAGA']

In [4]:
neg = np.array([np.random.choice(['A', 'C', 'G', 'T'], num_sample, p=np.array([1,1,1,1])/4)
                for i in range(seq_length)]).transpose()
[''.join(x) for x in neg[1:10,:]]

['GAGGCGAGATTCATACGATATGTGGTCCACAGCGCCGATT',
 'GGTCCTCCGAGTTGGAAACGTGGCCGCATCGTAGCCGCGG',
 'GATGGTGCGACTCCTAAGACTTGAGTGAACTTACGGACCT',
 'ATCCTAAACCTTAGCTACTATACGTACGGATTTAGGCCAG',
 'TTCTGGTAGCCAGCAACCTGCACACTATAAAAGGTAGATA',
 'AGTGAGTATCCCTTCACCATAATCGAAGATGACTAAAAAG',
 'TAAGCAGTTTATGCTAATGCGCAGAGCAGCAAAAACCGCC',
 'AACCGGCCCGGTCATAACTAAGGTGTGAAGGCGAAGGGAG',
 'GCCTGGCGAAGTCCGTAGGACACAGTACATTAAGAGTCTG']

In [5]:
pos_tensor = np.zeros(list(pos.shape) + [4])
neg_tensor = np.zeros(list(neg.shape) + [4])

base_dict = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

#naive one-hot encoding
for row in range(num_sample):
    for col in range(seq_length):
        pos_tensor[row,col,base_dict[pos[row,col]]] = 1
        neg_tensor[row,col,base_dict[neg[row,col]]] = 1

print('Positive sample matrix shape: {}'.format(pos.shape))
# this should be a 3D tensor with shape: (samples, steps, input_dim)
print('Positive sample tensor shape: {}'.format(pos_tensor.shape))

X = np.vstack((pos_tensor, neg_tensor))
y = np.concatenate((np.ones(num_sample), np.zeros(num_sample)))

print('Training set shape: {}'.format(X.shape))
print('Training set label shape: {}'.format(y.shape))
print('\nOne-hot encoding looks like:\n {}'.format(X[0,0:10,:]))

Positive sample matrix shape: (1000, 40)
Positive sample tensor shape: (1000, 40, 4)
Training set shape: (2000, 40, 4)
Training set label shape: (2000,)

One-hot encoding looks like:
 [[ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]]


In [6]:
#here comes the deep learning part
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Flatten, Dropout
from keras.activations import relu
from keras.layers.pooling import MaxPooling1D
from keras.optimizers import SGD

model = Sequential()
model.add(Convolution1D(1, 19, border_mode='same', input_shape=(seq_length, 4), activation='relu'))

#sanity check for dimensions
print('Shape of the output of first layer: {}'.format(model.predict_on_batch(pos_tensor[0:32,:,:]).shape))

Using Theano backend.


Shape of the output of first layer: (32, 40, 1)


In [7]:
#model.add(MaxPooling1D(pool_length=4))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])

hist = model.fit(X, y, validation_split=0.2, nb_epoch=4)  # starts training

Train on 1600 samples, validate on 400 samples
Epoch 1/4
1600/1600 [==============================] - 0s - loss: 0.4677 - acc: 0.7144 - val_loss: 0.6643 - val_acc: 0.7600
Epoch 2/4
1600/1600 [==============================] - 0s - loss: 0.2960 - acc: 0.8988 - val_loss: 0.4519 - val_acc: 0.9725
Epoch 3/4
1600/1600 [==============================] - 0s - loss: 0.2375 - acc: 0.9363 - val_loss: 0.3447 - val_acc: 0.9850
Epoch 4/4
1600/1600 [==============================] - 0s - loss: 0.1883 - acc: 0.9500 - val_loss: 0.2761 - val_acc: 0.9850


In [8]:
#have a look at the filter
convlayer = model.layers[0]
weights = convlayer.get_weights()[0].transpose().squeeze()
print('Convolution parameter shape: {}'.format(weights.shape))

Convolution parameter shape: (19, 4)


In [9]:
num2seq = ['A','C','G','T']

''.join([num2seq[np.argmax(weights[i,:])] for i in range(weights.shape[0])])

'AAGGAAGGGAGGGAGGAAG'